# Define bounding box around each aufeis feature of interest

In [5]:
# %%bash
# # test to see what GRIT looks like (thought about this instead of SWORD)

# # subset the GRIT river network to a the eastern north slope of AK
# cd ../
# ogr2ogr -f GPKG GRITv06_reaches_NorthSlopeAK_EPSG4326.gpkg GRITv06_reaches_simple_NA_EPSG4326.gpkg -spat -151.524048 67.205182 -140.139127 70.222115 -spat_srs EPSG:4326

# # for now, grit seems overly complex... don't want to track along messy centerline that have breaks around aufeis ...
# # might also be good to look at merit hydro

pj_obj_create: Open of /opt/conda/share/proj failed
pj_obj_create: Open of /opt/conda/share/proj failed


In [1]:
import os
import math
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from pyproj import Transformer
from netCDF4 import Dataset
from scipy.spatial import cKDTree
from collections import defaultdict

In [2]:
# ---------- paths & parameters ----------
sword_nc_path = "../Workspace/na_sword_v16.nc"
aufeis_shp_path = "../Workspace/aufeis_locations.shp"
out_shp_path = "../Workspace/polygon_aufeis.shp"
target_crs = "EPSG:32606"   # UTM zone 6N
scale_maxwidth = 1.0        # possible scalar for max_width

# Subset bbox in lon/lat EPSG:4326 
lon_min, lat_min, lon_max, lat_max = -151.524048, 67.205182, -140.139127, 70.222115

In [3]:
# ---------- Utility funcs ----------
def var_from_group(nc_obj, groupname, varname):
    """Return numpy array for a variable inside a group."""
    return nc_obj.groups[groupname].variables[varname][:]

def make_rotated_rectangle_from_segment(pt1, pt2, width):
    """
    Construct a rotated rectangle polygon aligned with segment from pt1->pt2.
    pt1, pt2 are (x,y) tuples in projected units.
    width is total width in same units.
    Returns shapely Polygon.
    """
    x1, y1 = pt1
    x2, y2 = pt2
    dx = x2 - x1
    dy = y2 - y1
    seg_len = math.hypot(dx, dy)
    if seg_len == 0:
        w2 = max(1.0, width / 2.0)
        return Point(x1, y1).buffer(w2, cap_style=2)
    angle = math.atan2(dy, dx)  # direction of segment
    perp_x = -math.sin(angle)
    perp_y = math.cos(angle)
    half_w = width / 2.0
    p1 = (x1 + perp_x * half_w, y1 + perp_y * half_w)
    p2 = (x2 + perp_x * half_w, y2 + perp_y * half_w)
    p3 = (x2 - perp_x * half_w, y2 - perp_y * half_w)
    p4 = (x1 - perp_x * half_w, y1 - perp_y * half_w)
    return Polygon([p1, p2, p3, p4, p1])


In [12]:
# ---------- Load aufeis features ----------
aufeis = gpd.read_file(aufeis_shp_path)
aufeis = aufeis.to_crs(target_crs)
print(f"Loaded {len(aufeis)} aufeis features. CRS: {aufeis.crs}")

Loaded 5 aufeis features. CRS: EPSG:32606


In [13]:
# ---------- Open netCDF and subset by bbox ----------
nc = Dataset(sword_nc_path, mode="r")

# Read node lon/lat and node ids
node_lon = var_from_group(nc, "nodes", "x")[:]       # degrees east
node_lat = var_from_group(nc, "nodes", "y")[:]       # degrees north
node_id_all = var_from_group(nc, "nodes", "node_id")[:]

# Mask nodes inside bbox
mask_nodes = (
    (node_lon >= lon_min) & (node_lon <= lon_max) &
    (node_lat >= lat_min) & (node_lat <= lat_max)
)
node_idx_sub = np.nonzero(mask_nodes)[0]
print(f"Nodes in bbox: {len(node_idx_sub)} / {node_lon.size}")

# Subset node arrays
node_id = node_id_all[node_idx_sub].astype(int)
node_lon = node_lon[node_idx_sub].astype(float)
node_lat = node_lat[node_idx_sub].astype(float)
node_max_width = var_from_group(nc, "nodes", "max_width")[:][node_idx_sub].astype(float)

# Read centerlines (global arrays) and subset by bbox
cl_x = var_from_group(nc, "centerlines", "x")[:]   # degrees east
cl_y = var_from_group(nc, "centerlines", "y")[:]   # degrees north
cl_id = var_from_group(nc, "centerlines", "cl_id")[:]  # int per point
cl_node_id_2d = var_from_group(nc, "centerlines", "node_id")[:]  # shape (num_domains, num_points)

# mask centerline points in bbox
mask_cl = (cl_x >= lon_min) & (cl_x <= lon_max) & (cl_y >= lat_min) & (cl_y <= lat_max)
cl_idx_sub = np.nonzero(mask_cl)[0]
print(f"Centerline points in bbox: {len(cl_idx_sub)} / {cl_x.size}")

# Subset centerline arrays
cl_x_sub = cl_x[cl_idx_sub].astype(float)
cl_y_sub = cl_y[cl_idx_sub].astype(float)
cl_id_sub = cl_id[cl_idx_sub].astype(int)
cl_node_id_sub = cl_node_id_2d[:, cl_idx_sub]  # keep domain dimension


Nodes in bbox: 21126 / 1683122
Centerline points in bbox: 128111 / 10304258


In [7]:
# ---------- Transform lon/lat -> UTM (target_crs) ----------
transformer = Transformer.from_crs("EPSG:4326", target_crs, always_xy=True)
nodes_x_utm, nodes_y_utm = transformer.transform(node_lon, node_lat)
nodes_xy = np.vstack([nodes_x_utm, nodes_y_utm]).T

cl_x_utm, cl_y_utm = transformer.transform(cl_x_sub, cl_y_sub)

Transforming coordinates to EPSG:32606 ...


In [14]:
# ---------- Build mapping node_id -> ordered list of (cl_id, xutm, yutm) ----------
CL_FILL_INT = -9999
num_domains, num_points_subset = cl_node_id_sub.shape
cl_by_node = defaultdict(list)

for d in range(num_domains):
    node_ids_for_domain = cl_node_id_sub[d, :]
    valid_mask = (node_ids_for_domain != CL_FILL_INT)
    valid_indices = np.nonzero(valid_mask)[0]
    for idx in valid_indices:
        p = int(idx)
        nid = int(node_ids_for_domain[p])
        # find the corresponding cl_id index in the subset arrays
        cl_idx_global = cl_idx_sub[p]  # p indexes into cl_idx_sub's order
        # Use the cl_id_sub order: the arrays cl_x_utm/cl_y_utm are aligned with cl_id_sub
        cl_val = int(cl_id_sub[p]) if cl_id_sub[p] != CL_FILL_INT else None
        if cl_val is None:
            continue
        cl_by_node[nid].append((cl_val, float(cl_x_utm[p]), float(cl_y_utm[p])))

# Sort each node's centerline points by cl_id
for nid in list(cl_by_node.keys()):
    cl_by_node[nid].sort(key=lambda t: t[0])

print(f"Nodes with centerline points in subset: {len(cl_by_node)}")


Nodes with centerline points in subset: 21166


In [15]:
# ---------- KDTree for snapping ----------
kdt = cKDTree(nodes_xy)

# Map node_id array index for quick lookup (node id -> index in nodes_xy arrays)
nodeid_to_index = {int(nid): i for i, nid in enumerate(node_id)}


In [16]:
# ---------- Iterate aufeis, snap, build polygon ----------
polys = []
attrs = []

for idx, row in aufeis.reset_index().iterrows():
    # row has 'index' (original index) and geometry + length_m etc.
    orig_index = int(row['index'])
    length_m = float(row['length_m'])
    geom = row.geometry
    if not isinstance(geom, Point):
        geom = geom.centroid
    px, py = geom.x, geom.y

    # nearest node (in UTM)
    dist, node_idx_local = kdt.query([px, py])
    node_idx_local = int(node_idx_local)
    snapped_node_id = int(node_id[node_idx_local])
    node_w = float(node_max_width[node_idx_local]) * scale_maxwidth
    if math.isnan(node_w) or node_w <= 0:
        # fallback to width variable from nodes group
        try:
            width_all = var_from_group(nc, "nodes", "width")[:]
            # need to subset same way
            width_sub = width_all[node_idx_sub]
            w_fallback = float(width_sub[node_idx_local])
            node_w = w_fallback if (not math.isnan(w_fallback) and w_fallback > 0) else 1.0
        except Exception:
            node_w = 1.0

    # Get centerline point list for snapped_node_id
    cl_list = cl_by_node.get(snapped_node_id, [])

    if len(cl_list) >= 2:
        # Build LineString from ordered cl_list
        cl_coords = [(t[1], t[2]) for t in cl_list]
        ln = LineString(cl_coords)
        # project aufeis point onto ln (both are UTM)
        proj_dist = ln.project(Point(px, py))
        half = length_m / 2.0
        start_d = max(0.0, proj_dist - half)
        end_d = min(ln.length, proj_dist + half)
        p_start = ln.interpolate(start_d)
        p_end = ln.interpolate(end_d)
        rect = make_rotated_rectangle_from_segment((p_start.x, p_start.y), (p_end.x, p_end.y), node_w)
    else:
        # fallback: compute rectangle using node's two extreme cl points (try reading cl_ids from nodes/cl_ids)
        # If no centerline, fallback to small buffered square around node coordinate.
        node_xy = nodes_xy[node_idx_local]
        rect = Point(node_xy[0], node_xy[1]).buffer(max(1.0, node_w/2.0), cap_style=2)

    polys.append(rect)
    attrs.append({
        "aufeis_index": orig_index,
        "snapped_node_id": snapped_node_id,
        "node_max_width": node_w,
        "length_m": length_m
    })


In [17]:
# ---------- Build GeoDataFrame and save ----------
out_gdf = gpd.GeoDataFrame(attrs, geometry=polys, crs=target_crs)

# merge original aufeis attributes by original index
aufeis_reset = aufeis.reset_index().rename(columns={"index": "aufeis_index"})
out_gdf = out_gdf.merge(aufeis_reset.drop(columns="geometry"), on="aufeis_index", how="left")

# save as shapefile
out_gdf.to_file(out_shp_path, driver="ESRI Shapefile")

print("Wrote", len(out_gdf), "polygons to", out_shp_path)

# close netCDF
nc.close()

Wrote 5 polygons to ../Workspace/polygon_aufeis.shp
